# Beer Data File Clean Up

In [1]:
#Import Dependencies
import csv
import pandas as pd
import requests
import json
from config import gkey
from pprint import pprint
import gmaps

In [2]:
#Import Beer Review Data
beer_data=pd.read_csv('../beer_reviews.csv', encoding = "ISO-8859-1")

#Preview the data
beer_data.head(10)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
5,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159
6,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
7,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159
8,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159
9,1075,Caldera Brewing Company,1285632924,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159


In [3]:
#Group the data by beer name, brewery, and beer type
beer = beer_data.groupby(["beer_name","brewery_name","beer_style"])

# Sort Data by the Beer's Average Review Values and Total Review Counts

In [4]:
#Create a data frame for average ratings based on individual beers
beer_df = pd.DataFrame(beer.mean())
ind_beer_rating_df = beer_df.reset_index(["brewery_name", "beer_style"])

#Change the id column back to an integer for merging later
ind_beer_rating_df.beer_beerid = ind_beer_rating_df.beer_beerid.astype(int)

#Display the data frame
ind_beer_rating_df

,brewery_name,beer_style,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
beer_name,,,,,,,,,,,
! (Old Ale),NÃ¤rke Kulturbryggeri AB,Old Ale,10902.0,1.271638e+09,4.000000,4.500000,4.000000,4.000000,4.500000,8.20,57645
"""100""",The Covey Restaurant & Brewery,Belgian Dark Ale,13338.0,1.254407e+09,4.000000,4.000000,4.000000,4.000000,4.400000,10.00,49236
"""100"" Pale Ale",Aviator Brewing Company,American IPA,18635.0,1.245965e+09,4.000000,4.000000,3.500000,4.000000,4.000000,6.60,50696
"""12"" Belgian Golden Strong Ale",Flossmoor Station Restaurant & Brewery,Belgian Strong Pale Ale,612.0,1.245585e+09,4.000000,3.500000,4.250000,4.750000,4.500000,9.00,44353
"""33"" Export",Baltika-Baku,Dortmunder / Export Lager,13640.0,1.200595e+09,3.000000,3.000000,3.000000,3.166667,3.000000,4.80,30080
"""4"" Horse Oatmeal Stout",Thoroughbreds Grill & Brewing,Oatmeal Stout,9819.0,1.126683e+09,4.000000,3.500000,4.000000,3.250000,3.250000,NaN,25570
"""400"" Ale",Williamsburg AleWerks,American Brown Ale,14952.0,1.201561e+09,3.333333,3.833333,3.916667,3.750000,3.444444,NaN,36909
"""50"" Golden Anniversary Lager",Mia & Pia's Pizzeria And Brewhouse,American Pale Lager,10434.0,1.100831e+09,4.000000,3.500000,5.000000,3.500000,3.500000,4.00,20819
"""76"" Anniversary Ale",Bootlegger's Brewery,Old Ale,17271.0,1.260115e+09,4.000000,4.166667,3.666667,4.000000,4.166667,7.60,51029


In [5]:
#Create a datafame to hold the total amount of reviews for each beer
beer_review_count = beer_data.groupby(['beer_name', 'beer_beerid']).count()
beer_review_count = beer_review_count[['review_overall']]
beer_review_count = beer_review_count.reset_index()
beer_review_count.head(10)

,beer_name,beer_beerid,review_overall
0,! (Old Ale),57645,1
1,"""100""",49236,5
2,"""100"" Pale Ale",50696,1
3,"""12"" Belgian Golden Strong Ale",44353,2
4,"""33"" Export",30080,3
5,"""4"" Horse Oatmeal Stout",25570,2
6,"""400"" Ale",36909,18
7,"""50"" Golden Anniversary Lager",20819,1
8,"""76"" Anniversary Ale",51029,3
9,"""76"" Anniversary Ale With English Hops",55510,1


In [14]:
#Merge average review dataframe and review count dataframe
beer_merged_data = pd.merge(ind_beer_rating_df, beer_review_count, on='beer_beerid', how='outer')
beer_merged_data = beer_merged_data.rename(columns={'review_overall_x': 'average_review', 'review_overall_y': 'review_count'})

#Preview Dataframe
beer_merged_data.head()

,brewery_name,beer_style,brewery_id,review_time,average_review,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,beer_name,review_count
0,NÃ¤rke Kulturbryggeri AB,Old Ale,10902.0,1.271638e+09,4.0,4.5,4.00,4.000000,4.5,8.2,57645,! (Old Ale),1.0
1,The Covey Restaurant & Brewery,Belgian Dark Ale,13338.0,1.254407e+09,4.0,4.0,4.00,4.000000,4.4,10.0,49236,"""100""",5.0
2,Aviator Brewing Company,American IPA,18635.0,1.245965e+09,4.0,4.0,3.50,4.000000,4.0,6.6,50696,"""100"" Pale Ale",1.0
3,Flossmoor Station Restaurant & Brewery,Belgian Strong Pale Ale,612.0,1.245585e+09,4.0,3.5,4.25,4.750000,4.5,9.0,44353,"""12"" Belgian Golden Strong Ale",2.0
4,Baltika-Baku,Dortmunder / Export Lager,13640.0,1.200595e+09,3.0,3.0,3.00,3.166667,3.0,4.8,30080,"""33"" Export",3.0


In [10]:
#Sort the merged dataframe by review count
beer_sorted_data = beer_merged_data.sort_values('review_count', ascending=False)
beer_sorted_data.head()

,brewery_name,beer_style,brewery_id,review_time,average_review,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,beer_name,review_count
947,Dogfish Head Brewery,American Double / Imperial IPA,10099.0,1.212149e+09,4.145745,4.213526,4.192097,4.181763,4.325988,9.0,2093,90 Minute IPA,3290.0
42967,North Coast Brewing Co.,Russian Imperial Stout,112.0,1.218958e+09,4.174221,4.198650,4.373995,4.231758,4.342173,9.0,412,Old Rasputin Russian Imperial Stout,3111.0
53469,Sierra Nevada Brewing Co.,American IPA,140.0,1.211116e+09,4.168667,4.082333,4.225167,4.076000,4.190000,6.8,1904,Sierra Nevada Celebration Ale,3000.0
60848,"Bell's Brewery, Inc.",American IPA,287.0,1.221429e+09,4.329729,4.266496,4.153959,4.138746,4.318915,7.0,1093,Two Hearted Ale,2728.0
56517,Stone Brewing Co.,American Double / Imperial IPA,147.0,1.215671e+09,4.161612,4.341531,4.179179,4.183432,4.351331,7.7,4083,Stone Ruination IPA,2704.0


In [15]:
#Create bins based on the fibonacci sequence in order to remove beers with a lower review count
bins = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]
review_count_slices = ['1', '2', '3', '4-5', '6-8', '9-13', '14-21', '22-34', '35-55', '56-89', '90-144', '145-233', '234-377', '378-610', '611-987', '988-1597', '1598-2584', '2585 and up']
beer_sorted_data['review_count_slices'] = pd.cut(beer_sorted_data['review_count'], bins, labels=review_count_slices)
beer_sorted_data.head()

,brewery_name,beer_style,brewery_id,review_time,average_review,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,beer_name,review_count,review_count_slices
947,Dogfish Head Brewery,American Double / Imperial IPA,10099.0,1.212149e+09,4.145745,4.213526,4.192097,4.181763,4.325988,9.0,2093,90 Minute IPA,3290.0,2585 and up
42967,North Coast Brewing Co.,Russian Imperial Stout,112.0,1.218958e+09,4.174221,4.198650,4.373995,4.231758,4.342173,9.0,412,Old Rasputin Russian Imperial Stout,3111.0,2585 and up
53469,Sierra Nevada Brewing Co.,American IPA,140.0,1.211116e+09,4.168667,4.082333,4.225167,4.076000,4.190000,6.8,1904,Sierra Nevada Celebration Ale,3000.0,2585 and up
60848,"Bell's Brewery, Inc.",American IPA,287.0,1.221429e+09,4.329729,4.266496,4.153959,4.138746,4.318915,7.0,1093,Two Hearted Ale,2728.0,2585 and up
56517,Stone Brewing Co.,American Double / Imperial IPA,147.0,1.215671e+09,4.161612,4.341531,4.179179,4.183432,4.351331,7.7,4083,Stone Ruination IPA,2704.0,2585 and up


In [16]:
#Create a dataframe to be able to see how many beers are in each bin
review_1 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '1', :]
review_1_count = review_1['review_count'].count()
review_2 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '2', :]
review_2_count = review_2['review_count'].count()
review_3 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '3', :]
review_3_count = review_3['review_count'].count()
review_4_5 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '4-5', :]
review_4_5_count = review_4_5['review_count'].count()
review_6_8 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '6-8', :]
review_6_8_count = review_6_8['review_count'].count()
review_9_13 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '9-13', :]
review_9_13_count = review_9_13['review_count'].count()
review_14_21 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '14-21', :]
review_14_21_count = review_14_21['review_count'].count()
review_22_34 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '22-34', :]
review_22_34_count = review_22_34['review_count'].count()
review_35_55 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '35-55', :]
review_35_55_count = review_35_55['review_count'].count()
review_56_89 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '56-89', :]
review_56_89_count = review_56_89['review_count'].count()
review_90_144 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '90-144', :]
review_90_144_count = review_90_144['review_count'].count()
review_145_233 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '145-233', :]
review_145_233_count = review_145_233['review_count'].count()
review_234_377 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '234-377', :]
review_234_377_count = review_234_377['review_count'].count()
review_378_610 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '378-610', :]
review_378_610_count = review_378_610['review_count'].count()
review_611_987 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '611-987', :]
review_611_987_count = review_611_987['review_count'].count()
review_988_1597 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '988-1597', :]
review_988_1597_count = review_988_1597['review_count'].count()
review_1598_2584 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '1598-2584', :]
review_1598_2594_count = review_1598_2584['review_count'].count()
review_2585_andup = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '2585 and up', :]
review_2585_andup_count = review_2585_andup['review_count'].count()



review_count_df = pd.DataFrame({'review_count': [review_1_count, review_2_count, review_3_count, review_4_5_count, review_6_8_count, review_9_13_count, review_14_21_count, review_22_34_count, review_35_55_count, review_56_89_count, review_90_144_count, review_145_233_count, review_234_377_count, review_378_610_count, review_611_987_count, review_988_1597_count, review_1598_2594_count, review_2585_andup_count]})
review_count_df = review_count_df.rename(index={0 : '1', 1 : '2', 2 : '3', 3 : '4-5', 4 : '6-8', 5 : '9-13', 6 : '14-21', 7 : '22-34', 8 : '35-55',
                                                9: '56-89', 10: '90-144', 11: '145-233', 12: '234-377', 13: '378-610', 14: '611-987', 15: '988-1597', 16: '1598-2584', 17: '2585 and up'})
review_count_df


,review_count
1,23805
2,10447
3,5785
4-5,6373
6-8,4609
9-13,3688
14-21,2827
22-34,2191
35-55,1743
56-89,1422


In [ ]:
#Remove beers that have less than 9 total reviews
final_count_df = beer_sorted_data.loc[(beer_sorted_data['review_count_slices'] != '1') &
                                      (beer_sorted_data['review_count_slices'] != '2') &
                                      (beer_sorted_data['review_count_slices'] != '3') &
                                      (beer_sorted_data['review_count_slices'] != '4-5') &
                                      (beer_sorted_data['review_count_slices'] != '6-8'), :]
#Save to csv
final_count_df.to_csv('beer_review_data_cleaned.csv')

# Sort Data by Beer Style

In [34]:
#Creat a dataframe for beer count by style
beer_type = beer_df.reset_index()
beer_type_df = beer_type.groupby(['beer_style']).count()
beer_type_df = beer_type_df[['beer_name']]
beer_type_df = beer_type_df.sort_values(['beer_name'], ascending=False)
beer_type_df = beer_type_df.reset_index()
beer_type_df = beer_type_df.rename(columns={'beer_name': 'beer_count'})

#Preview dataframe
beer_type_df.head(10)

,beer_style,beer_count
0,American IPA,3588
1,American Pale Ale (APA),3320
2,American Amber / Red Ale,2489
3,American Porter,2181
4,Fruit / Vegetable Beer,1833
5,Hefeweizen,1713
6,American Double / Imperial IPA,1546
7,English Bitter,1486
8,Euro Pale Lager,1479
9,American Stout,1418


In [40]:
#Create a dataframe for review count by stle
beer_review_style = beer_data.groupby(['beer_style']).count()
beer_review_style = beer_review_style[['review_overall']]
beer_review_style = beer_review_style.sort_values(["review_overall"],ascending=False)
beer_review_style = beer_review_style.reset_index()
beer_review_style = beer_review_style.rename(columns={'review_overall': 'review_count'})

#Preview the dataframe
beer_review_style.head(10)

,beer_style,review_count
0,American IPA,117586
1,American Double / Imperial IPA,85977
2,American Pale Ale (APA),63469
3,Russian Imperial Stout,54129
4,American Double / Imperial Stout,50705
5,American Porter,50477
6,American Amber / Red Ale,45751
7,Belgian Strong Dark Ale,37743
8,Fruit / Vegetable Beer,33861
9,American Strong Ale,31945


In [41]:
#Create a dataframe for the average overall rating by style
style_means = beer_data.groupby(['beer_style']).review_overall.mean()
beer_style_means_df = pd.DataFrame(style_means)
beer_style_means_df = beer_style_means_df.sort_values(['review_overall'], ascending=False)
beer_style_means_df = beer_style_means_df.reset_index()
beer_style_means_df = beer_style_means_df.rename(columns={'review_overall': 'review_avg'})

beer_style_means_df.head(10)

,beer_style,review_avg
0,American Wild Ale,4.093262
1,Gueuze,4.086287
2,Quadrupel (Quad),4.071630
3,Lambic - Unblended,4.048923
4,American Double / Imperial Stout,4.029820
5,Russian Imperial Stout,4.023084
6,Weizenbock,4.007969
7,American Double / Imperial IPA,3.998017
8,Flanders Red Ale,3.992722
9,Rye Beer,3.981737


In [43]:
beer_style_df = pd.merge(beer_type_df, beer_review_style, on='beer_style', how='outer')
beer_style_df = pd.merge(beer_style_df, beer_style_means_df, on='beer_style', how='outer')

beer_style_df.head()

,beer_style,beer_count,review_count,review_avg
0,American IPA,3588,117586,3.965221
1,American Pale Ale (APA),3320,63469,3.852306
2,American Amber / Red Ale,2489,45751,3.781010
3,American Porter,2181,50477,3.895735
4,Fruit / Vegetable Beer,1833,33861,3.415124


In [44]:
#Save to csv
beer_style_df.to_csv('beer_style_data_cleaned.csv')

# Sort Data by Brewery

In [46]:
#Create a dataframe by brewery and the average review for the beers they make
brew_rating = beer_data.groupby(["brewery_name"])
average_rating = round(brew_rating['review_overall'].mean(), 2)
brewery_avg_rating_df = pd.DataFrame(average_rating)
brewery_avg_rating_df = brewery_avg_rating_df.reset_index()
brewery_avg_rating_df = brewery_avg_rating_df.rename(columns={'brewery_name': 'Brewery'})

#Preview the dataframe
brewery_avg_rating_df.head()

,Brewery,review_overall
0,'t Hofbrouwerijke,3.70
1,(512) Brewing Company,4.07
2,10 Barrel Brewing Co.,3.73
3,1516 Brewing Company,4.13
4,16 Mile Brewing Company,3.57
